In [3]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor,BaggingClassifier

In [4]:
df=pd.read_csv('newdf.csv')
columns=df.columns

FileNotFoundError: File b'newdf.csv' does not exist

In [3]:
columns

Index(['user_id', 'cities', 'states', 'postal_codes', 'alcohol',
       'bike_parking', 'accepts_credit_cards', 'caters', 'drivethru',
       'goodforkids', 'hastv', 'noise_level', 'outdoor_seating', 'delivery',
       'goodforgroups', 'pricerange', 'reservations', 'table_service',
       'takeout', 'wifi', 'star', 'postalcount', 'statescount', 'citycount',
       'newMeanCity', 'newMeanState', 'newMeanPostalCode', 'newVarCity',
       'newVarState', 'newVarPostalCode'],
      dtype='object')

# Regression


In [5]:
y=df['star']
take=['newMeanState','newMeanPostalCode','newMeanCity','postalcount','statescount','citycount','newVarState','newVarCity','newVarPostalCode']# only take continuos features
X=df[take]
X=X.fillna(0)

In [6]:
#Normalize
scaler=StandardScaler().fit(X)
X=pd.DataFrame(scaler.transform(X))
X.columns=take


/anaconda3/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
model= GradientBoostingRegressor(n_estimators=500, learning_rate=0.01,
    max_depth=2, random_state=0, loss='ls')

In [9]:
model.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [10]:
ytemp=model.predict(X_test)

In [11]:
newscore=[]
for i in ytemp:
    a=round(i,2)
    dif=a-int(i)
    if(dif<0.25):
        newscore.append(int(i))
    elif(0.25<dif<0.75):
        newscore.append(int(i)+0.5)
    elif(0.75<dif):
        newscore.append(int(i)+1)
        
    

In [12]:
count=0
for i in range(len(newscore)):
    if(newscore[i]==y_test.iloc[i]):
        count+=1
print(count/len(newscore))        

0.1670190673966396


# Classifier

In [13]:
take=columns[5:-9]
take

Index(['bike_parking', 'accepts_credit_cards', 'caters', 'drivethru',
       'goodforkids', 'hastv', 'noise_level', 'outdoor_seating', 'delivery',
       'goodforgroups', 'pricerange', 'reservations', 'table_service',
       'takeout', 'wifi', 'star'],
      dtype='object')

In [15]:
#Encode into categorial data
from sklearn.preprocessing import LabelEncoder
df1=pd.DataFrame()
for i in take:
   
    LE = LabelEncoder()
    print(i)
    df1[i]= LE.fit_transform(df[i])

bike_parking
accepts_credit_cards
caters
drivethru
goodforkids
hastv
noise_level
outdoor_seating
delivery
goodforgroups
pricerange
reservations
table_service
takeout
wifi
star


In [16]:
#Normalize continuos features
scaler=StandardScaler().fit(df[columns[-9:]])
X1=pd.DataFrame(scaler.transform(df[columns[-9:]]))
X1.columns=columns[-9:]

/anaconda3/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
#Create X and y
X=df1
for i in columns[-9:]:
    X[str(i)]=X1[i]
X=X.fillna(0)

y=LE.fit_transform(df['star'])


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
#Bagging with Knn 
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(),
                             max_samples=0.5, max_features=0.5)
bagging.fit(X_train,y_train)
print(bagging.score(X_test,y_test))

0.9975946416015986


In [20]:
#Adaboost
from sklearn.ensemble import AdaBoostClassifier
model=AdaBoostClassifier(n_estimators=200,learning_rate=0.35)
model.fit(X_train,y_train)
print(model.score(X_test,y_test))

0.8950708655589683


In [43]:
#Voting
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
clf1 = DecisionTreeClassifier(max_depth=7)
clf2 = KNeighborsClassifier(n_neighbors=10)

clf4=BaggingClassifier(KNeighborsClassifier(),
                             max_samples=0.5, max_features=0.5)
clf5=AdaBoostClassifier(n_estimators=200,learning_rate=0.35)


In [44]:
clf1 = clf1.fit(X_train,y_train)
clf1.score(X_test,y_test)

1.0

In [45]:
clf2 = clf2.fit(X_train,y_train)
clf2.score(X_test,y_test)

0.8820079191799578

In [46]:
from sklearn.ensemble import GradientBoostingClassifier
clf3=GradientBoostingClassifier(n_estimators=150, learning_rate=0.5)
clf3.fit(X_train,y_train)
clf3.score(X_test,y_test)

1.0

In [47]:
clf4 = clf4.fit(X_train,y_train)
clf4.score(X_test,y_test)

0.9854753358250379

In [48]:
clf5 = clf5.fit(X_train,y_train)
clf5.score(X_test,y_test)

0.6180475890907745

In [49]:
eclf = VotingClassifier(estimators=[('dt', clf1), ('knn', clf2),('GBC',clf3),('bagging',clf4),("adaboost",clf5)],
                        voting='soft', weights=[2,1,2,2,0.5])


eclf.fit(X_train,y_train)
eclf.score(X_test,y_test)

1.0